In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
base_dir="/kaggle/input/divvytripdata-1/"

df=pd.read_csv(base_dir+'202005-divvy-tripdata.csv')
df_2=pd.read_csv(base_dir+'202006-divvy-tripdata.csv')
df_3=pd.read_csv(base_dir+'202007-divvy-tripdata.csv')
df_4=pd.read_csv(base_dir+'202008-divvy-tripdata.csv')
df_5=pd.read_csv(base_dir+'202009-divvy-tripdata.csv')
df_6=pd.read_csv(base_dir+'202010-divvy-tripdata.csv')
df_7=pd.read_csv(base_dir+'202011-divvy-tripdata.csv')
df_8=pd.read_csv(base_dir+'202012-divvy-tripdata.csv')
df_9=pd.read_csv(base_dir+'202101-divvy-tripdata.csv')
df_10=pd.read_csv(base_dir+'202102-divvy-tripdata.csv')
df_11=pd.read_csv(base_dir+'202103-divvy-tripdata.csv')
df_12=pd.read_csv(base_dir+'202105-divvy-tripdata.csv')
df_13=pd.read_csv(base_dir+'202104-divvy-tripdata.csv')

In [ ]:
df=df.append([df_2 , df_3 ,df_4 , df_5, df_6, df_7 , df_8 ,df_9 , df_10,df_11 , df_12,df_13],ignore_index=True)

In [ ]:
df.head()

In [ ]:
df.shape

## Lest Apply univariate analysis 

In [ ]:
# Check For duplicated Data
df.duplicated().sum()

In [ ]:
df.info()

In [ ]:
df['rideable_type'].value_counts()

In [ ]:
sns.countplot(data=df,x='rideable_type')

In [ ]:
df['member_casual'].value_counts()

In [ ]:
sns.set(rc={'figure.figsize': [7, 5]}, font_scale=1.3)
sns.countplot(data=df,x='member_casual')

In [ ]:
df['started']=pd.to_datetime(df['started_at'],format='%m/%d/%Y %H:%M',errors='coerce')

In [ ]:
df['started_at']=df['started']
df.drop('started',axis=1,inplace=True)

In [ ]:
df['ended']=pd.to_datetime(df['ended_at'],format='%m/%d/%Y %H:%M',errors='coerce')

In [ ]:
df['ended_at']=df['ended']
df.drop('ended',axis=1,inplace=True)

In [ ]:
#Sumation of NULL values
df.isnull().sum()

In [ ]:
#percentage of NULL values
df.isnull().mean()*100

In [ ]:
df.drop(['Unnamed: 13','s'],inplace=True,axis=1)

## Lest Apply bivariate analysis

In [ ]:
df[['start_station_name','end_station_name']]

In [ ]:
df[df['started_at']==df['ended_at']].shape[0]

## feature engineering


In [ ]:
from geopy.distance import great_circle
def get_des(x):
        loc1=(x['start_lat'],x['start_lng'])
        loc2=(x['end_lat'],x['end_lng'])
        return great_circle(loc1,loc2).kilometers
df['destace(KM)']=df.apply(get_des,axis=1)

In [ ]:
df[['destace(KM)']]

In [ ]:
df.describe()[['destace(KM)']]

In [ ]:
sns.set(rc={'figure.figsize': [5, 5]}, font_scale=1.3)
sns.kdeplot(data=df,x='destace(KM)')

In [ ]:
sns.set(rc={'figure.figsize': [10, 5]}, font_scale=1.3)
sns.boxplot(data=df,x='destace(KM)')

In [ ]:
#prcentege of destace(KM) that = 0
df[df['destace(KM)']==0].shape[0]/df.shape[0]*100

In [ ]:
idx_of_zero_des=df[df['destace(KM)']==0].index
df.drop(index=idx_of_zero_des,axis=0,inplace=True)

In [ ]:
sns.set(rc={'figure.figsize': [10, 10]}, font_scale=1.3)
sns.displot(data=df,x='destace(KM)',)

## Handel OutLayers For destace(KM)

In [ ]:
!pip install -q datasist

In [ ]:
from datasist.structdata import detect_outliers
outliers_indices = detect_outliers(data=df, n=0, features=['destace(KM)'])
len(outliers_indices)

In [ ]:
#prcentege of outliers at destace(KM) that = 0
df[df.index.isin(outliers_indices)].shape[0]/df.shape[0]*100

In [ ]:
idx=df.index.isin(outliers_indices)

In [ ]:
# df.drop(index=idx,axis=0,inplace=True)

In [ ]:
sns.set(rc={'figure.figsize': [10, 10]}, font_scale=1.3)
sns.kdeplot(data=df,x='destace(KM)',shade=True)

In [ ]:
df['year']=df['started_at'].dt.year
df['year'].unique()

In [ ]:
df['hour']=df['started_at'].dt.hour
df['day']=df['started_at'].dt.day_name()
df['month']=df['started_at'].dt.month

## Lets divide the day to piriods 

In [ ]:
periods={'Morning':[i for i in range(1,14)],'Afternoon':[i for i in range(14,20)],'Evening':[i for i in range(20,22)],
     'Night':[i for i in range(22,25)]}

In [ ]:
def get_periods(x):
    if x in periods['Morning']:
        return 'Morning'
    elif x in periods['Afternoon']:
        return 'Afternoon'
    elif x in periods['Evening']:
        return 'Evening'
    else:
        return 'Night'

In [ ]:
df['period']=df['hour'].apply(get_periods)
df[['period']]

In [ ]:
df['period'].value_counts()

In [ ]:
sns.set(rc={'figure.figsize': [10, 4]}, font_scale=1.3)
sns.countplot(data=df,x='period')

# Lets Answer Business Questions

## Questions

### 1- For each rideable_type calculate the sum of kms ridead
### 2- For each member_casual calculate the sum of kms ridead
### 3- For each rideable_type for each member_casual calculate the sum of kms driven
### 4- How many Rides in every start_station_name and visualize the top 10
### 5- How many Rides driven in all Days of this month and visualize them descening
### 6- How many Rides driven in all period of the day and visualiaze them descending
### 7- How many KMS driven in all period of the day and visualiaze them descending
### 8- For each rideable_type for each member_casual which has most driven count
### 9- Which start station and end station with maximam number of Rides between them
### 10- Which start station and end station with maximam number of kms between them
### 11-How many Rides driven in all Months of year and visualize the top 4 month descening
### 12-How many kms driven in all Months of year and visualize the top 4 month descening and then divide them    by days of week
### 13-How many Rides driven in every year

### 14- For each month for each week calculate the sum of kms driven (12)


### 1- For each rideable_type calculate the sum of kms ridead


In [ ]:
sns.set(rc={'figure.figsize': [9, 5]}, font_scale=1.3)
sns.barplot(data=df,x='rideable_type',y='destace(KM)',estimator=sum)

In [ ]:
df.groupby('rideable_type').sum()[['destace(KM)']]

### 2- For each member_casual calculate the sum of kms ridead


In [ ]:
sns.set(rc={'figure.figsize': [9, 5]}, font_scale=1.3)
sns.barplot(data=df,x='member_casual',y='destace(KM)',estimator=sum)

### 3- For each rideable_type for each member_casual calculate the sum of kms driven


In [ ]:
df.pivot_table(index='rideable_type',columns='member_casual',values='destace(KM)',aggfunc=sum)

In [ ]:
sns.barplot(data=df,x='rideable_type',y='destace(KM)',hue='member_casual',estimator=sum)

### 4- How many Rides in every start_station and visualize the top 10


In [ ]:
df['start_station_name'].value_counts()

In [ ]:
Index_oftop_10_start_station_name=df['start_station_name'].value_counts().sort_values(ascending=False).head(10).index
Data_of_top_10_start_station_name=df[df['start_station_name'].isin(Index_oftop_10_start_station_name)]
Data_of_top_10_start_station_name.head()

In [ ]:
sns.countplot(data=Data_of_top_10_start_station_name,y='start_station_name',order=Index_oftop_10_start_station_name)

### 5- How many Rides driven in all Days and visualize them desceningdf['day'].value_counts()

In [ ]:
df['day'].value_counts()

In [ ]:
sns.countplot(data=df,x='day',order=df['day'].value_counts().index)

### 6- How many Rides driven in all period of the day and visualiaze them descending

In [ ]:
df['period'].value_counts()

In [ ]:
sns.countplot(data=df,x='period',order=df['period'].value_counts().index)

### 7- How many KMS driven in all period of the day and visualiaze them descending


In [ ]:
sns.barplot(data=df,x='period',y='destace(KM)',estimator=sum,order=df['period'].value_counts().index)

In [ ]:
df.groupby('period').sum()['destace(KM)']

### 8- For each rideable_type for each member_casual which has most driven count


In [ ]:
df['dumy']=1
df.pivot_table(columns='rideable_type',index='member_casual',values='dumy',aggfunc=sum)

In [ ]:
sns.countplot(data=df,x='rideable_type',hue='member_casual')

### 9- Which start station and end station with maximam number of Rides between them

In [ ]:
pivot_Rides= df.pivot_table(columns='start_station_name',index='end_station_name',values='dumy',aggfunc=sum)
pivot_Rides

In [ ]:
pivot_Rides.max().sort_values(ascending=False)

In [ ]:
max_start_station_km=pivot_Rides['Ellis Ave & 60th St'].sort_values(ascending=False).index[0]
max_start_station_km

In [ ]:
max_end_station_km=pivot_Rides[max_start_station_km].sort_values(ascending=False).index[0]
max_end_station_km

In [ ]:
df[(df['start_station_name']==max_start_station_km)&(df['end_station_name']==max_end_station_km)]

### 11-How many Rides driven in all Months of year and visualize the top 4 month descening


In [ ]:
sns.countplot(data=df,y='month',order=df['month'].value_counts().sort_values(ascending=False).index)

In [ ]:
df['month'].value_counts()

In [ ]:
#visualize the top 4 month descening
top_4_month_descening_index=df['month'].value_counts().sort_values(ascending=False).head(4).index

In [ ]:
top_4_month_descening_data=df[df['month'].isin(top_4_month_descening_index)]
top_4_month_descening_data.head()

In [ ]:
sns.countplot(data=top_4_month_descening_data,y='month',
              order=top_4_month_descening_data['month'].value_counts().sort_values(ascending=False).index)

### 12-How many kms driven in all Months of year and visualize the top 4 month descening and then divide them by days of week

In [ ]:
df.groupby('month').sum()[['destace(KM)']]

In [ ]:
top_4_month_km_idx=df.groupby('month').sum()['destace(KM)'].sort_values(ascending=False).head(4).index

In [ ]:
top_4_month_km_data=df[df['month'].isin(top_4_month_km_idx)]
top_4_month_km_data.head()

In [ ]:
sns.barplot(data=top_4_month_km_data,y='month',x='destace(KM)' ,
                           order=top_4_month_km_data['month'].value_counts().sort_values(ascending=False).index)

In [ ]:
sns.barplot(data=top_4_month_km_data,x='month',y='destace(KM)' ,hue='day',
                           order=top_4_month_km_data['month'].value_counts().sort_values(ascending=False).index)

### 13-How many Rides driven in every year

In [ ]:
sns.countplot(data=df,x='year')

In [ ]:
sns.countplot(data=df,y='month',hue='year',
              order=df['month'].value_counts().sort_values(ascending=False).index)

### 14- For each month for each week calculate the sum of kms driven (Q-12)


In [ ]:
df.pivot_table(columns='month',index='day',values='destace(KM)',aggfunc=sum)

In [ ]:
sns.heatmap(df.pivot_table(columns='month',index='day',values='destace(KM)',aggfunc=sum))